# **Vector DB - FAISS DB**

**Note - This script executed in Google Colab**

- Different **Vector DB -** **Chroma DB, Pinecone, Weaviate, FAISS(Meta), Redis, etc now mangodb added vector db/search**
- **Croma DB(Local_cloud) , FAISS** are **open source local db**, we can download to local and try.
- **Weaiate, Pinecone** are **cloud based db**, we need to **take subscription**, but it provides initial free credits, we can create only 1 cluster. if we **dont want to save our private data** there, then we should use Chrome db/FAISS

## **Terminology:**
- **faiss-cpu** Db **pip installed**, Then called via **langchain's vectore_stores**
- Here we used hugging faces's embedding -**sentence-transformers** - **This framework generates embeddings for each input sentence**
- **Chunking/Chunk_size:** In document/datset we will have more no of tokens, but word embedding LLM models will have **token size /token_limitation** like 4k Tokens etc, So to accomodate to that size, we **split our data as chunks**
- **Chunk_overlap =50:** It takes **50 token behind from previous chunk** while creating next chunk

## **Below steps followed:**
- 1st **download some document** - Here used some pdf and saved in colab notebook's folder pdf
- Then **split that into chunks**
- Then import **openai embedding or hugging face embedding model** or some other embedding which converts **tokens/text to vector**
- Only for **Chroma db** , Create **persist_directory** to create **schema/db** in local, Foe **FAISS** no need to create db. It creates itself in its memory
- Then use **Chroma/vectore db library** and pass
    - **document which conveted to chunks**  
    - **embedding model name**
- This converts **chunk to vectors/embedding**, which will be **saved inside in memory not in any db folder(ChromaDB)**
- Then we need to **USE this vector_db** which we just now created**vector_db**
- Then use **as_retriever** to **read vector db** and **do  symantic search on this**
- Then this **symantic/similarity search** will give **K=4 relavant answers**, that along **with user Q** we will **feed to LLM** to provide **meaningfull response on that Q**.
- We can use **langchain's chain operation** - **RetrivalQA** for this
- We can set this # of relevant answer by setting **search_kwargs ={k:2}**
by using Chroma library
- Here **VectorDB does similarity search based on user Q** but **LLM just structure the VectorDB response and gives as output**. LLM wont do anything else. **Its also called RAG**
- This **RetrievalQA** passes Q to Vector db **retriever** and then passes this O/P with Q to llm model to do **summarization** internally
- We can use langchain's chain operation - **RetrivalQA** or **load_qa_chain** for this


In [5]:
!pip -q install langchain pypdf sentence-transformers==2.2.2 -q
!pip install openai tiktoken faiss-cpu -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.4/262.4 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 41.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 46.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.0 MB/s eta 0:00:00


## **1. Read the Document**
- Create directory pdfs and keep pdf file here, which will be used to created DB
- This pdf folder creating inside colab env,so it will deleted once session completes

In [6]:
!mkdir pdfs

mkdir: cannot create directory ‘pdfs’: File exists


### **Extract the Text from the PDF's**

In [14]:
from langchain.document_loaders import PyPDFDirectoryLoader

In [20]:
loader = PyPDFDirectoryLoader("pdfs")
data = loader.load()
data[:1]

[Document(page_content='Retrieval-Augmented Generation for Large Language Models: A Survey\nYunfan Gao1,Yun Xiong2,Xinyu Gao2,Kangxiang Jia2,Jinliu Pan2,Yuxi Bi3,Yi\nDai1,Jiawei Sun1,Qianyu Guo4,Meng Wang3and Haofen Wang1,3∗\n1Shanghai Research Institute for Intelligent Autonomous Systems, Tongji University\n2Shanghai Key Laboratory of Data Science, School of Computer Science, Fudan University\n3College of Design and Innovation, Tongji University\n4School of Computer Science, Fudan University\nAbstract\nLarge Language Models (LLMs) demonstrate\nsignificant capabilities but face challenges such\nas hallucination, outdated knowledge, and non-\ntransparent, untraceable reasoning processes.\nRetrieval-Augmented Generation (RAG) has\nemerged as a promising solution by incorporating\nknowledge from external databases. This enhances\nthe accuracy and credibility of the models, particu-\nlarly for knowledge-intensive tasks, and allows for\ncontinuous knowledge updates and integration of\ndomai

### Split the whole document to chunks
- split that into chunks with **chunk_size=500, chunk_overlap=20** using **RecursiveCharacterTextSplitter**

In [21]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [22]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
text_chunks = text_splitter.split_documents(data)

In [23]:
len(text_chunks)

281

In [24]:
text_chunks[2]

Document(page_content='domain-specific information. RAG synergistically\nmerges LLMs’ intrinsic knowledge with the vast,\ndynamic repositories of external databases. This\ncomprehensive review paper offers a detailed\nexamination of the progression of RAG paradigms,\nencompassing the Naive RAG, the Advanced RAG,\nand the Modular RAG. It meticulously scrutinizes\nthe tripartite foundation of RAG frameworks,\nwhich includes the retrieval , the generation and\nthe augmentation techniques. The paper highlights', metadata={'source': 'pdfs/RAG_LLM_Pdf.pdf', 'page': 0})

## **2. Creating Vector DB**

- Then import **openai embedding or hugging face embedding model** or some other embedding which converts **tokens/text to vector**
- Only for **Chroma db** , Create **persist_directory** to create **schema/db** in local, Foe **FAISS** no need to create db. It creates itself in its memory
- Then use **Chroma/vectore db library** and pass
    - **document which conveted to chunks to vector**  
    - **embedding model name**
    - **persist directory**
- This converts **chunk to vectors/embedding**, which will be **saved inside db folder**

### **Initialize Embedding**

- Used Hugging face embedding - **sentence-transformers/all-MiniLM-L6-v2**

In [26]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

### **Create Vector DB**

In [28]:
#Do pip install FAISS, but call this FAISS via langchain.vectorstores
from langchain.vectorstores import FAISS


#Initialize FAISS by passing text which converted as chunks, embedding model and schema name
vectorstore=FAISS.from_documents(text_chunks, embeddings)

### Use the Loaded Vector DB and do similarity check

In [30]:
query = "What is RAG?"

docs = vectorstore.similarity_search(query, k=3)
docs

[Document(page_content='able contexts, and forecast potential future developments.\nOur contributions are as follows:\n• We present a thorough and systematic review of the\nstate-of-the-art RAG, delineating its evolution through\nparadigms including naive RAG, advanced RAG, and\nmodular RAG. This review contextualizes the broader\nscope of RAG research within the landscape of LLMs.\n• We identify and discuss the central technologies integral\nto the RAG process, specifically focusing on the aspects', metadata={'source': 'pdfs/RAG_LLM_Pdf.pdf', 'page': 1}),
 Document(page_content='the state-of-the-art technologies embedded in\neach of these critical components, providing a\nprofound understanding of the advancements in\nRAG systems. Furthermore, this paper introduces\nthe metrics and benchmarks for assessing RAG\nmodels, along with the most up-to-date evaluation\nframework. In conclusion, the paper delineates\nprospective avenues for research, including the\nidentification of challenges

In [31]:
len(docs)

3

## **3. Sementic Search**
- Then use **as_retriever** to read vector db and do **symantic search** on this
- Then this symantic/similarity search will give K=4 relavant answers, that along with user Q we will feed to LLM to provide meaningfull response on that Q.
- We can use langchain's chain operation - **RetrivalQA** or **load_qa_chain** for this

In [34]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

import os
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

from langchain.llms import OpenAI

#Import OpenAI -LLM
llm = OpenAI()

### Use  **RetrievalQA** from langchain and to QA

In [ ]:
### Set Retrival argument **search_kwargs={"k": 3}**

In [35]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(llm=llm,
                                 chain_type="stuff",
                                 retriever=vectorstore.as_retriever())

In [37]:
query = "What is RAG?"
print("\n",qa.run(query))


 
RAG stands for "Retrieve and Generate," and it refers to a research paradigm that combines information retrieval and context-aware generation techniques to create more advanced language models. This paper provides a review of the state-of-the-art RAG, including its evolution through different paradigms and the central technologies integral to the RAG process. It also introduces metrics and benchmarks for evaluating RAG models and discusses potential future developments in RAG research.


# **END**